In [55]:
import torch
import numpy as np  

In [67]:
torch.cuda.empty_cache()

In [56]:
best_model = torch.load('./exp/run_20231206_relu_clean/best-valid-loss.pt')

In [57]:
best_model['best_valid_epoch']

28

In [58]:
del best_model

In [7]:
last_epoch_model = torch.load('./exp/run_20231206_relu_clean/epoch-30.pt')

In [8]:
last_epoch_model['best_valid_epoch']

28

In [5]:
last_epoch_model = torch.load('./exp/run_20231208_leakyRelu_clean/epoch-30.pt')

In [6]:
last_epoch_model['best_valid_epoch']

23

In [12]:
last_epoch_model = torch.load('./exp/run_20231206_doubleswish_clean/epoch-30.pt')

In [13]:
last_epoch_model['best_valid_epoch']

23

In [17]:
last_epoch_model = torch.load('./exp/run_20231209_tanh_clean/epoch-13.pt')

In [18]:
last_epoch_model['best_valid_epoch']

6

In [23]:
last_epoch_model = torch.load('./exp/run_20231208_singleswish_clean/epoch-25.pt')

In [24]:
last_epoch_model['best_valid_epoch']

6

In [9]:
last_epoch_model = torch.load('./exp/run_20231210_gelu_clean/epoch-30.pt')

In [11]:
last_epoch_model['best_valid_epoch']

17

In [60]:
last_epoch_model = torch.load('./exp/run_20231210_elu_clean/epoch-30.pt')

In [61]:
last_epoch_model['best_valid_epoch']

6

In [62]:
last_epoch_model = torch.load('./exp/run_20231211_selu_clean/epoch-30.pt')

In [63]:
last_epoch_model['best_valid_epoch']

6

In [64]:
last_epoch_model = torch.load('./exp/run_20231211_softplus_clean/epoch-30.pt')

In [65]:
last_epoch_model['best_valid_epoch']

26

In [66]:
del last_epoch_model

In [19]:
last_epoch_model.keys()

dict_keys(['model', 'optimizer', 'scheduler', 'grad_scaler', 'sampler', 'model_avg', 'best_train_loss', 'best_valid_loss', 'best_train_epoch', 'best_valid_epoch', 'batch_idx_train', 'log_interval', 'reset_interval', 'valid_interval', 'feature_dim', 'subsampling_factor', 'encoder_dim', 'nhead', 'dim_feedforward', 'num_encoder_layers', 'beam_size', 'reduction', 'use_double_scores', 'model_warm_step', 'env_info', 'world_size', 'master_port', 'tensorboard', 'num_epochs', 'start_epoch', 'start_batch', 'exp_dir', 'lang_dir', 'initial_lr', 'lr_batches', 'lr_epochs', 'att_rate', 'num_decoder_layers', 'seed', 'print_diagnostics', 'save_every_n', 'keep_last_k', 'average_period', 'use_fp16', 'activation_type', 'full_libri', 'mini_libri', 'manifest_dir', 'max_duration', 'bucketing_sampler', 'num_buckets', 'concatenate_cuts', 'duration_factor', 'gap', 'on_the_fly_feats', 'shuffle', 'drop_last', 'return_cuts', 'num_workers', 'enable_spec_aug', 'spec_aug_time_warp_factor', 'enable_musan', 'input_stra

In [21]:
last_epoch_model['env_info']

{'k2-version': '1.24.4',
 'k2-build-type': 'Release',
 'k2-with-cuda': True,
 'k2-git-sha1': '59aef5e3a52a43990f78ad33f4839f0a1680358a',
 'k2-git-date': 'Thu Nov 23 09:08:57 2023',
 'lhotse-version': '1.17.0.dev+git.b869488.clean',
 'torch-version': '2.1.0+cu121',
 'torch-cuda-available': True,
 'torch-cuda-version': '12.1',
 'python-version': '3.1',
 'icefall-git-branch': 'master',
 'icefall-git-sha1': '7c682ec-dirty',
 'icefall-git-date': 'Sat Dec 9 06:50:50 2023',
 'icefall-path': '/home/ipsoct4/project/icefall_w4995_mathml',
 'k2-path': '/home/ipsoct4/anaconda3/envs/kaldi/lib/python3.11/site-packages/k2/__init__.py',
 'lhotse-path': '/home/ipsoct4/anaconda3/envs/kaldi/lib/python3.11/site-packages/lhotse/__init__.py',
 'hostname': 'instance-4t4',
 'IP address': '10.182.0.2'}

In [ ]:
del last_epoch_model

torch.cuda.empty_cache()

## load TF events

In [1]:
import tensorflow as tf


2023-12-13 18:13:08.170181: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 18:13:08.170383: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 18:13:08.358651: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 18:13:09.322808: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 18:13:15.637463: W tensorflow/comp

In [19]:
from tensorflow.core.util import event_pb2
import numpy as np

def parse_event(event):
    return event_pb2.Event.FromString(event)

# Replace with the path to your event file
event_file = "./exp/run_20231211_softplus_clean/tensorboard/events.out.tfevents.1702344251.instance-4t4.1178832.0"

steps_per_epoch = 1156  # Example value

step_times = []
epoch_durations = []

for event in tf.compat.v1.train.summary_iterator(event_file):
    if event.WhichOneof('what') == 'summary':
        step_times.append(event.wall_time)

# Calculate epoch durations
for i in range(len(step_times) - steps_per_epoch):
    if i % steps_per_epoch == 0:
        epoch_index = i // steps_per_epoch
        if 25 <= epoch_index <= 30:
            epoch_duration = step_times[i + steps_per_epoch] - step_times[i]
            epoch_durations.append(epoch_duration)

# Calculate average time per epoch for epochs 25 to 30
if epoch_durations:
    average_duration = sum(epoch_durations) / len(epoch_durations)
    print(f"Average time per epoch (for epochs 25 to 30): {average_duration} seconds")
else:
    print("No epoch data found for the specified range.")


Average time per epoch (for epochs 25 to 30): 176.22150373458862 seconds


In [ ]:
# find all directories with tensorboard files, within each directory, find the latest event file, and parse it to get the epoch durations
import os
import glob
import re
import datetime

computing_time = {}

# def calculate_avg_epoch_duration(directory='./exp/'):
    # get the name of all subdirectories
    # for each subdirectory name, prune the name to get the experiment name

directory='./exp/'
for subdir in os.listdir(directory):
    exp_name = subdir[13:]
    # print(subdir)
    # get the latest event file
    event_file = max(glob.glob(directory+ subdir + '/tensorboard/events.out.tfevents.*'), key=os.path.getctime)
    # parse the event file to get the epoch durations
    epoch_times = {}

    for event in tf.compat.v1.train.summary_iterator(event_file):
        for value in event.summary.value:
            if value.tag == "train/epoch":
                epoch_number = int(value.simple_value)
                if epoch_number not in epoch_times:
                    epoch_times[epoch_number] = event.wall_time

    
    last_available_epoch = min(max(epoch_times.keys()), 30)

    if min(epoch_times.keys()) > 25:
        first_available_epoch = min(epoch_times.keys())
    elif min(epoch_times.keys()) <= 6:
        first_available_epoch = min(epoch_times.keys())
    else:
        first_available_epoch = last_available_epoch - 5


    average_duration_raw = (epoch_times[last_available_epoch] - epoch_times[first_available_epoch]) / (last_available_epoch - first_available_epoch) 

    average_duration_processed = str(datetime.timedelta(seconds=average_duration_raw))[:7]

    computing_time[exp_name] = average_duration_processed
    
    # return computing_time

# calculate_avg_epoch_duration(directory='./exp')


In [124]:
computing_time

{'gelu_clean': '0:32:16',
 'relu_clean': '0:31:19',
 'selu_clean': '0:32:11',
 'singleswish_clean': '0:33:00',
 'tanh_clean': '0:32:02',
 'leakyRelu_clean': '0:32:13',
 'elu_clean': '0:32:03',
 'softplus_all_activation_clean': '0:32:37',
 'softplus_clean': '0:32:27',
 'doubleswish_clean': '0:33:35'}